In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from datasets import load_dataset

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
## Here we should read the data from a CSV file or any other source.
# then visualize it using pandas ot matplot to gain insights from it.
# Download dataset from Kaggle and extract/Unzip it.
#!kaggle datasets download -d kazanova/sentiment140

import requests, zipfile, io
url = 'http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
import shutil
shutil.rmtree('/root/.cache/huggingface', ignore_errors=True)
!pip install --upgrade datasets fsspec --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [ ]:
# We load the dataset
# We load the dataset
header_list = ["label", "id", "date", "query", "user", "text"]
df = pd.read_csv('training.1600000.processed.noemoticon.csv',
                 encoding = "ISO-8859-1", names=header_list)

# Limit the dataset size
DATASET_SIZE=10000
#df = df[:DATASET_SIZE]

# Show the first few rows
df

,label,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df['label'] = df['label'].map(lambda x: 0 if x == 0 else 1 if x == 4 else x)


In [ ]:
df

,label,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
from datasets import Dataset
from tokenizers import BertWordPieceTokenizer

# Convert the pandas DataFrame to a datasets Dataset
dataset = Dataset.from_pandas(df)

# Initialize a tokenizer
tokenizer = BertWordPieceTokenizer()

# Customize training using the dataset's text column
tokenizer.train(files="/content/text.txt", vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model(".", "tokenizer")

['./tokenizer-vocab.txt']

In [2]:
from transformers import BertTokenizerFast

# Load the pretrained tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

d:\Anaconda2\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
tokens=[]
sentences=[]
# Load your custom vocabulary file
with open(r"C:\Users\hp\Downloads\text.txt", "r", encoding="utf-8") as f:
      for sentence in f:
        sentences.append(sentence.strip())
        tokens.append(word_tokenize(sentence))

In [4]:
sentences[:10]

['awake early damn project super nervous science one mine gunna lame dont wan na fail',
 'watching church online id half hour late time got',
 'hillsong',
 'stafford train station watched em train pull away heading southbound wcml',
 'thanks everyone follow friday',
 'waffle house like 5am proceeded drop living room rug grit hard clean youre drunk',
 'afternoon going see hannah montana movie like mileycyrus hair',
 'rinnathecat watched news u look great gold printed maxi dress wonder shoe u r wearing',
 'fly beautiful love',
 'finished sarah baylas first birthday party']

In [5]:
tokens[:10]

[['awake',
  'early',
  'damn',
  'project',
  'super',
  'nervous',
  'science',
  'one',
  'mine',
  'gunna',
  'lame',
  'dont',
  'wan',
  'na',
  'fail'],
 ['watching', 'church', 'online', 'id', 'half', 'hour', 'late', 'time', 'got'],
 ['hillsong'],
 ['stafford',
  'train',
  'station',
  'watched',
  'em',
  'train',
  'pull',
  'away',
  'heading',
  'southbound',
  'wcml'],
 ['thanks', 'everyone', 'follow', 'friday'],
 ['waffle',
  'house',
  'like',
  '5am',
  'proceeded',
  'drop',
  'living',
  'room',
  'rug',
  'grit',
  'hard',
  'clean',
  'youre',
  'drunk'],
 ['afternoon',
  'going',
  'see',
  'hannah',
  'montana',
  'movie',
  'like',
  'mileycyrus',
  'hair'],
 ['rinnathecat',
  'watched',
  'news',
  'u',
  'look',
  'great',
  'gold',
  'printed',
  'maxi',
  'dress',
  'wonder',
  'shoe',
  'u',
  'r',
  'wearing'],
 ['fly', 'beautiful', 'love'],
 ['finished', 'sarah', 'baylas', 'first', 'birthday', 'party']]

In [6]:
words=[word for les in tokens for word in les]

In [7]:
import re

In [8]:
# Load tokenizer vocab keys just once and make it a set (faster lookup)
existing_vocab = set(tokenizer.get_vocab().keys())

In [9]:
Unique_words = [word for word in words if word not in existing_vocab]

In [10]:
len(Unique_words)

3002675

In [11]:
Unique_words= Unique_words[:2400000]

In [12]:
# Add new tokens to the tokenizer
added_tokens = tokenizer.add_tokens(Unique_words)


In [13]:

# Save the extended tokenizer
tokenizer.save_pretrained("./tokenizer1-extended")

('./tokenizer1-extended\\tokenizer_config.json',
 './tokenizer1-extended\\special_tokens_map.json',
 './tokenizer1-extended\\vocab.txt',
 './tokenizer1-extended\\added_tokens.json',
 './tokenizer1-extended\\tokenizer.json')

In [14]:
from transformers import BertTokenizerFast, BertForSequenceClassification
# Load model (e.g. binary classification: num_labels=2)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [15]:
model.resize_token_embeddings(len(tokenizer))


Embedding(672636, 768, padding_idx=0)

In [18]:
print("Embedding size:", model.get_input_embeddings().weight.size())
print("Tokenizer vocab size:", len(tokenizer))


Embedding size: torch.Size([672636, 768])
Tokenizer vocab size: 672636


In [21]:
import torch

text = "Bad News"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    print("Predicted class:", predictions.item())


Predicted class: 1


In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load from local
tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\hp\Downloads\tokenizer1-extended")

In [24]:
# @title SelfAttention
class SelfAttention(nn.Module):
  """  Multi-head self attention layer. """

  def __init__(self, k, heads=8, dropout=0.1):
    """
    Initiates the following attributes:
    to_keys: Transforms input to k x k*heads key vectors
    to_queries: Transforms input to k x k*heads query vectors
    to_values: Transforms input to k x k*heads value vectors
    unify_heads: combines queries, keys and values to a single vector

    Args:
      k: Integer
        Size of attention embeddings
      heads: Integer
        Number of attention heads

    Returns:
      Nothing
    """
    super().__init__()
    self.k, self.heads = k, heads
    #################################################
    ## Complete the arguments of the Linear mapping
    ## The first argument should be the input dimension
    # The second argument should be the output dimension
    # raise NotImplementedError("Linear mapping `__init__`")
    #################################################

    self.to_keys = nn.Linear(k, k * heads , bias=False)
    self.to_queries = nn.Linear(k, k * heads, bias=False)
    self.to_values = nn.Linear(k, k * heads, bias=False)
    self.unify_heads = nn.Linear(k * heads, k)
    self.attention = DotProductAttention(dropout)

  def forward(self, x):
    """
    Implements forward pass of self-attention layer

    Args:
      x: Tensor
        Batch x t x k sized input

    Returns:
      unify_heads: Tensor
        Self-attention based unified Query/Value/Key tensors
    """
    b, t, k = x.size()
    h = self.heads

    # We reshape the queries, keys and values so that each head has its own dimension
    queries = self.to_queries(x).view(b, t, h, k)
    keys = self.to_keys(x).view(b, t, h, k)
    values = self.to_values(x).view(b, t, h, k)

    out = self.attention(queries, keys, values, b, h, t, k)

    return self.unify_heads(out)

In [25]:
# @title DotProductAttention
class DotProductAttention(nn.Module):
  """ Scaled dot product attention. """

  def __init__(self, dropout, **kwargs):
    """
    Constructs a Scaled Dot Product Attention Instance.

    Args:
      dropout: Integer
        Specifies probability of dropout hyperparameter

    Returns:
      Nothing
    """
    super(DotProductAttention, self).__init__(**kwargs)
    self.dropout = nn.Dropout(dropout)

  def calculate_score(self, queries, keys):
      """
      Compute the score between queries and keys.

      Args:
      queries: Tensor
        Query is your search tag/Question
        Shape of `queries`: (`batch_size`, no. of queries, head,`k`)
      keys: Tensor
        Descriptions associated with the database for instance
        Shape of `keys`: (`batch_size`, no. of key-value pairs, head, `k`)
      """
      return torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(queries.shape[-1])

  def forward(self, queries, keys, values, b, h, t, k):
    """
    Compute dot products. This is the same operation for each head,
    so we can fold the heads into the batch dimension and use torch.bmm
    Note: .contiguous() doesn't change the actual shape of the data,
    but it rearranges the tensor in memory, which will help speed up the computation
    for this batch matrix multiplication.
    .transpose() is used to change the shape of a tensor. It returns a new tensor
    that shares the data with the original tensor. It can only swap two dimensions.

    Args:
      queries: Tensor
        Query is your search tag/Question
        Shape of `queries`: (`batch_size`, no. of queries, head,`k`)
      keys: Tensor
        Descriptions associated with the database for instance
        Shape of `keys`: (`batch_size`, no. of key-value pairs, head, `k`)
      values: Tensor
        Values are returned results on the query
        Shape of `values`: (`batch_size`, head, no. of key-value pairs,  `k`)
      b: Integer
        Batch size
      h: Integer
        Number of heads
      t: Integer
        Number of keys/queries/values (for simplicity, let's assume they have the same sizes)
      k: Integer
        Embedding size

    Returns:
      out: Tensor
        Matrix Multiplication between the keys, queries and values.
    """
    keys = keys.transpose(1, 2).contiguous().view(b * h, t, k)
    queries = queries.transpose(1, 2).contiguous().view(b * h, t, k)
    values = values.transpose(1, 2).contiguous().view(b * h, t, k)

    #################################################
    ## Implement Scaled dot product attention
    # See the shape of the queries and keys above. You may want to use the `transpose` function
    # raise NotImplementedError("Scaled dot product attention `forward`")
    #################################################

    # Matrix Multiplication between the keys and queries
    score = self.calculate_score(queries, keys)  # size: (b * h, t, t)
    softmax_weights = F.softmax(score, dim=2)  # row-wise normalization of weights

    # Matrix Multiplication between the output of the key and queries multiplication and values.
    out = torch.bmm(self.dropout(softmax_weights), values).view(b, h, t, k)  # rearrange h and t dims
    out = out.transpose(1, 2).contiguous().view(b, t, h * k)

    return out

In [26]:
import tqdm

In [27]:
# OR just use this cell from the course notebook XD
from transformers import  AutoModelForSequenceClassification
import statistics

bert_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           num_labels=2)

class MiniBERTClassifier(nn.Module):
    def __init__(self, encoder, num_classes):
        super(MiniBERTClassifier, self).__init__()
        self.encoder = encoder
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.encoder.config.hidden_size, num_classes)
        self.attention = SelfAttention(self.encoder.config.hidden_size)
        self.DotProductAttention=DotProductAttention(0.3)
        self.layers = nn.Sequential(
            nn.Linear(self.encoder.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        self.classifier = nn.Linear(self.encoder.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        # Only use the pooled output (representation of [CLS] token)
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.logits # Access logits directly from the AutoModelForSequenceClassification output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits


    def train_model(self, encoded, labels, epochs=10, learning_rate=5e-3):
        loss_function = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate)
        losses = []
        for epoch in range(epochs):
            input_ids = encoded['input_ids'].to(DEVICE)
            attention_mask = encoded['attention_mask'].to(DEVICE)
            y = labels.long().to(DEVICE)
            y = y.squeeze()
            y_logits = self(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_function(y_logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            if epoch % 1000 == 0:
                print(f"Epoch {epoch} loss is {loss.item()}")
        return losses

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [29]:
model= MiniBERTClassifier(bert_model, num_classes=2)

train_layers = ["classifier", "bert.pooler", "bert.encoder.layer.11"]  # add/remove layers here (use layer-name sub-strings)

for name, param in model.named_parameters():
  if any(x in name for x in train_layers):
    param.requires_grad = True
    print("FINE-TUNING -->", name)
  else:
    param.requires_grad = False
    print("FROZEN -->", name)

FROZEN --> encoder.bert.embeddings.word_embeddings.weight
FROZEN --> encoder.bert.embeddings.position_embeddings.weight
FROZEN --> encoder.bert.embeddings.token_type_embeddings.weight
FROZEN --> encoder.bert.embeddings.LayerNorm.weight
FROZEN --> encoder.bert.embeddings.LayerNorm.bias
FROZEN --> encoder.bert.encoder.layer.0.attention.self.query.weight
FROZEN --> encoder.bert.encoder.layer.0.attention.self.query.bias
FROZEN --> encoder.bert.encoder.layer.0.attention.self.key.weight
FROZEN --> encoder.bert.encoder.layer.0.attention.self.key.bias
FROZEN --> encoder.bert.encoder.layer.0.attention.self.value.weight
FROZEN --> encoder.bert.encoder.layer.0.attention.self.value.bias
FROZEN --> encoder.bert.encoder.layer.0.attention.output.dense.weight
FROZEN --> encoder.bert.encoder.layer.0.attention.output.dense.bias
FROZEN --> encoder.bert.encoder.layer.0.attention.output.LayerNorm.weight
FROZEN --> encoder.bert.encoder.layer.0.attention.output.LayerNorm.bias
FROZEN --> encoder.bert.encoder.

In [ ]:
model = MiniBERTClassifier(encoder=bert_model, num_classes=2)
model.train_model(encoded,labels)